# Film Development Time Prediction

End-to-end orchestration of the pipeline:

1. Data loading & exploration
2. Frozen train/val/test splits
3. Feature engineering walkthrough (slopes → latent vectors → aggregates)
4. 5-fold cross-validation with ExtraTreesRegressor
5. Final model: train on all trainval, evaluate on held-out test
6. Feature importance & error analysis

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.ensemble import ExtraTreesRegressor

from data_split import DataSplitter, prepare_data
from film_slopes import FilmSlopeEstimator, DevDilSlopeEstimator, DevDilutionSlopeEstimator
from latent_features import LatentFeatureBuilder
from aggregate_features import AggregateFeatureBuilder
from train_extratrees import build_features, smape, evaluate, DROP_COLS, TARGET

%matplotlib inline
plt.rcParams.update({'figure.dpi': 110, 'axes.spines.top': False, 'axes.spines.right': False})

## 1. Data Loading & Exploration

In [ ]:
print("=" * 60)
print("Filtering pipeline")
print("=" * 60)
df = prepare_data("film_data.csv")

print(f"""
Final dataset
  Rows        : {len(df):,}
  Films       : {df['Film'].nunique():,}
  Developers  : {df['Developer'].nunique():,}
  dev_dils    : {df['dev_dil'].nunique():,}
  Time range  : {df['35mm'].min():.1f} – {df['35mm'].max():.1f} min
  ISO range   : {df['iso'].min():.0f} – {df['iso'].max():.0f}
""")
df.head(3)

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(14, 4))

# Development time distribution
axes[0].hist(df['35mm'], bins=40, color='steelblue', edgecolor='white', linewidth=0.4)
axes[0].set_xlabel('Development time (min)')
axes[0].set_ylabel('Count')
axes[0].set_title('Development time distribution')
axes[0].axvline(df['35mm'].median(), color='tomato', linestyle='--', label=f"median {df['35mm'].median():.1f} min")
axes[0].legend(fontsize=9)

# ISO distribution
iso_counts = np.log2(df['iso']).round(1)
axes[1].hist(iso_counts, bins=30, color='seagreen', edgecolor='white', linewidth=0.4)
axes[1].set_xlabel('log₂(ISO)')
axes[1].set_ylabel('Count')
axes[1].set_title('ISO distribution (log₂ scale)')

# Top 15 films by row count
top_films = df['Film'].value_counts().head(15)
axes[2].barh(top_films.index[::-1], top_films.values[::-1], color='mediumpurple')
axes[2].set_xlabel('Row count')
axes[2].set_title('Top 15 films by row count')

plt.tight_layout()
plt.show()

In [ ]:
# Stops (push/pull) distribution
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

axes[0].hist(df['stops'], bins=40, color='darkorange', edgecolor='white', linewidth=0.4)
axes[0].set_xlabel('Stops from box ISO (log₂)')
axes[0].set_ylabel('Count')
axes[0].set_title('Push/pull distribution')
axes[0].axvline(0, color='black', linestyle=':', linewidth=1, label='box ISO')
axes[0].legend(fontsize=9)

# Time vs stops scatter (sample)
sample = df.sample(min(2000, len(df)), random_state=0)
sc = axes[1].scatter(sample['stops'], sample['35mm'], c=np.log2(sample['iso']),
                     cmap='plasma', alpha=0.35, s=8, linewidths=0)
plt.colorbar(sc, ax=axes[1], label='log₂(ISO)')
axes[1].set_xlabel('Stops from box ISO')
axes[1].set_ylabel('Development time (min)')
axes[1].set_title('Time vs push/pull (sample of 2000)')

plt.tight_layout()
plt.show()

## 2. Train/Val/Test Splits

In [ ]:
splitter = DataSplitter.load("splits.json")

print(f"Seed          : {splitter.seed}")
print(f"N folds       : {splitter.n_folds}")
print(f"Test fraction : {splitter.test_fraction}")
print(f"Test rows     : {len(splitter.test_indices):,}")
print(f"Trainval rows : {len(splitter.get_trainval_indices()):,}")
print()

summary = splitter.summary(df)
display(summary[['set', 'n_rows', 'pct_total', 'n_films', 'n_dev_dils',
                  'time_median', 'time_mean', 'time_std']].round(2))

In [ ]:
# Verify no leakage
test_set = set(splitter.test_indices)
all_folds = [set(f) for f in splitter.fold_indices]

for i, fold in enumerate(all_folds):
    assert len(test_set & fold) == 0, f"Test overlaps fold {i}"
for i in range(len(all_folds)):
    for j in range(i + 1, len(all_folds)):
        assert len(all_folds[i] & all_folds[j]) == 0, f"Fold {i} overlaps fold {j}"

all_assigned = test_set.copy()
for fold in all_folds:
    all_assigned |= fold
assert len(all_assigned) == len(df)

print("Split integrity checks passed:")
print("  No test/fold overlap")
print("  No fold/fold overlap")
print(f"  All {len(df):,} rows accounted for")

## 3. Feature Engineering Walkthrough (Fold 0)

All transformers are fit on the **training fold only** to prevent leakage.

### 3a. Slope Estimation

In [ ]:
train_idx, val_idx = splitter.get_fold(0)
df_train0 = df.loc[train_idx]
df_val0   = df.loc[val_idx]

# Fit all three slope estimators on training fold
film_est  = FilmSlopeEstimator().fit(df_train0)
devdil_est = DevDilSlopeEstimator(box_isos=film_est.box_isos).fit(df_train0)
dil_est   = DevDilutionSlopeEstimator().fit(df_train0)

print(f"FilmSlopeEstimator     : {len(film_est.film_slopes):,} films with specific slope  "
      f"(global={film_est.global_slope:.4f})")
print(f"DevDilSlopeEstimator   : {len(devdil_est.dev_dil_slopes):,} dev_dils with specific slope  "
      f"(global={devdil_est.global_slope:.4f})")
print(f"DevDilutionSlopeEstimator: {len(dil_est.dev_dilution_slopes):,} developers with dilution slope  "
      f"(global={dil_est.global_slope:.4f})")

In [ ]:
# Film slope summary — top 10 most-observed films
fs = film_est.summary().sort_values('n_devs', ascending=False)
print("Top 10 films by developer count:")
display(fs[['Film','slope','accel','se','r2_within','n_obs','n_devs','box_iso']].head(10).round(4))

In [ ]:
# Visualise film slope distribution
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

axes[0].hist(fs['slope'], bins=30, color='steelblue', edgecolor='white', linewidth=0.4)
axes[0].axvline(film_est.global_slope, color='tomato', linestyle='--',
                label=f'global slope={film_est.global_slope:.3f}')
axes[0].set_xlabel('Film push/pull slope β₁')
axes[0].set_ylabel('Count')
axes[0].set_title('Per-film push/pull slope distribution')
axes[0].legend(fontsize=9)

axes[1].scatter(fs['slope'], fs['accel'], alpha=0.5, s=20, color='seagreen')
axes[1].set_xlabel('Linear slope β₁')
axes[1].set_ylabel('Acceleration β₂')
axes[1].set_title('Slope vs acceleration per film')

plt.tight_layout()
plt.show()

In [ ]:
# Visualise a well-known film's push/pull curve
candidate_films = [f for f in ['Ilford HP5 Plus 400', 'Kodak Tri-X 400', 'Ilford Delta 400 Professional',
                                'Kodak T-Max 400'] if f in film_est.film_slopes]
if candidate_films:
    film_name = candidate_films[0]
    info = film_est.film_slopes[film_name]
    stops_range = np.linspace(-2, 3, 80)
    log_ratio = info['slope'] * stops_range + info['accel'] * stops_range**2

    # Raw data points
    film_rows = df_train0[df_train0['Film'] == film_name]

    fig, ax = plt.subplots(figsize=(8, 4))
    ax.scatter(film_rows['stops'], np.log(film_rows['35mm']), alpha=0.4, s=15,
               color='steelblue', label='training data (log time)')
    ax.plot(stops_range, log_ratio + np.log(film_rows['35mm'].median()), color='tomato',
            linewidth=2, label=f'quadratic fit (β₁={info["slope"]:.3f}, β₂={info["accel"]:.3f})')
    ax.set_xlabel('Stops from box ISO')
    ax.set_ylabel('log(time)')
    ax.set_title(f'Push/pull curve: {film_name}')
    ax.legend(fontsize=9)
    ax.axvline(0, color='black', linestyle=':', linewidth=1)
    plt.tight_layout()
    plt.show()
else:
    print("None of the target films found in fold 0 training data.")

### 3b. Matrix Factorization (Latent Features)

In [ ]:
lv = LatentFeatureBuilder(n_factors=30, reg=0.005)
lv_train0 = lv.fit_transform(df_train0)
lv_val0   = lv.transform(df_val0)

info = lv.summary()
print(f"Matrix shape  : {info['matrix_shape'][0]} dev_dils × {info['matrix_shape'][1]} film@ISO columns")
print(f"Matrix density: {info['density']:.1%}")
print(f"Filled cells  : {info['filled_cells']:,}")
print(f"Recon SMAPE   : {info['recon_smape']:.2f}%")
print(f"Recon R²      : {info['recon_r2']:.4f}")
print(f"Films with LV : {info['n_films_with_lv']}")
print(f"Dev_dils w/ LV: {info['n_dd_with_lv']}")
print(f"Global mean   : {info['global_mean_time']:.2f} min")
print(f"\nLV columns ({lv_train0.shape[1]}):")
print(', '.join(lv_train0.columns[:6].tolist()), '...',
      ', '.join(lv_train0.columns[-3:].tolist()))

In [ ]:
# Coverage on val
val_cov = lv_val0['lv_available'].mean()
print(f"Val LV coverage : {val_cov:.1%} ({lv_val0['lv_available'].sum()} / {len(lv_val0)} rows)")

# MF-only SMAPE on covered val rows
mask = lv_val0['lv_available']
if mask.sum() > 0:
    actual    = df_val0.loc[mask, '35mm'].values
    predicted = lv_val0.loc[mask, 'lv_pred_time'].values
    mf_smape  = np.mean(2 * np.abs(actual - predicted) / (np.abs(actual) + np.abs(predicted))) * 100
    mf_mae    = np.mean(np.abs(actual - predicted))
    print(f"MF-only val SMAPE: {mf_smape:.2f}%  MAE: {mf_mae:.2f} min  (on {mask.sum()} rows)")

### 3c. Aggregate Features

In [ ]:
agg = AggregateFeatureBuilder()
agg_train0 = agg.fit_transform(df_train0)
agg_val0   = agg.transform(df_val0)

s = agg.summary()
print(f"Entity levels   : {s['entity_levels']}  (film, film@ISO, dev_dil, developer)")
print(f"Stats per entity: {s['features_per_entity']}")
print(f"Row features    : {s['row_features']}")
print(f"Total features  : {s['total_features']}")
print(f"\nEntities seen in training:")
print(f"  Films     : {s['n_films']}")
print(f"  Film@ISO  : {s['n_film_iso']}")
print(f"  Dev_dils  : {s['n_dev_dils']}")
print(f"  Developers: {s['n_developers']}")

# Coverage
for prefix, label in [('film_agg', 'Film'), ('film_iso_agg', 'Film@ISO'),
                       ('dd_agg', 'Dev_dil'), ('dev_agg', 'Developer')]:
    col = f'{prefix}_count'
    cov = agg_val0[col].notna().mean() * 100
    print(f"  Val coverage {label:10s}: {cov:.1f}%")

### 3d. Combined Feature Matrix

In [ ]:
X_train0, y_train0, X_val0, y_val0, feature_names = build_features(df_train0, df_val0)

print(f"X_train shape : {X_train0.shape}")
print(f"X_val shape   : {X_val0.shape}")
print(f"\nNaN fraction per split:")
print(f"  train : {np.isnan(X_train0).mean():.2%}")
print(f"  val   : {np.isnan(X_val0).mean():.2%}")
print(f"\nFirst 10 feature names: {feature_names[:10]}")

## 4. Cross-Validation

In [ ]:
print("=" * 70)
print("5-Fold Cross-Validation — ExtraTreesRegressor")
print("=" * 70)

fold_results = []

for fold in range(splitter.n_folds):
    train_idx, val_idx = splitter.get_fold(fold)
    df_train = df.loc[train_idx]
    df_val   = df.loc[val_idx]

    t0 = time.time()
    X_train, y_train, X_val, y_val, feat_names = build_features(df_train, df_val)
    t_feat = time.time() - t0

    t0 = time.time()
    et = ExtraTreesRegressor(
        n_estimators=500, max_features=0.5,
        min_samples_leaf=2, random_state=42, n_jobs=-1,
    )
    et.fit(X_train, y_train)
    t_train = time.time() - t0

    y_pred_train = et.predict(X_train)
    y_pred_val   = et.predict(X_val)

    print(f"\nFold {fold}  (feat: {t_feat:.0f}s  train: {t_train:.0f}s  features: {X_train.shape[1]})")
    evaluate(y_train, y_pred_train, 'train')
    res = evaluate(y_val, y_pred_val, 'val')
    fold_results.append({'fold': fold, **res, 'n_val': len(y_val)})

print()
cv_df = pd.DataFrame(fold_results)
print("=" * 70)
print("CV Summary")
print("=" * 70)
for metric in ['smape', 'mae', 'mape', 'r2']:
    vals = cv_df[metric]
    print(f"  {metric:8s}  mean={vals.mean():.4f}  std={vals.std():.4f}  "
          f"min={vals.min():.4f}  max={vals.max():.4f}")

In [ ]:
# Visualise CV results
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

folds = cv_df['fold'].values

axes[0].bar(folds, cv_df['smape'], color='steelblue', alpha=0.8)
axes[0].axhline(cv_df['smape'].mean(), color='tomato', linestyle='--',
                label=f"mean {cv_df['smape'].mean():.2f}%")
axes[0].set_xticks(folds)
axes[0].set_xlabel('Fold')
axes[0].set_ylabel('SMAPE (%)')
axes[0].set_title('Validation SMAPE per fold')
axes[0].legend(fontsize=9)

axes[1].bar(folds, cv_df['mae'], color='seagreen', alpha=0.8)
axes[1].axhline(cv_df['mae'].mean(), color='tomato', linestyle='--',
                label=f"mean {cv_df['mae'].mean():.2f} min")
axes[1].set_xticks(folds)
axes[1].set_xlabel('Fold')
axes[1].set_ylabel('MAE (min)')
axes[1].set_title('Validation MAE per fold')
axes[1].legend(fontsize=9)

plt.tight_layout()
plt.show()

## 5. Final Model — Train on All Trainval, Evaluate on Test

In [ ]:
trainval_idx = splitter.get_trainval_indices()
test_idx     = splitter.test_indices

df_trainval = df.loc[trainval_idx]
df_test     = df.loc[test_idx]

t0 = time.time()
X_tv, y_tv, X_test, y_test, final_feature_names = build_features(df_trainval, df_test)
print(f"Feature build time: {time.time() - t0:.1f}s")

t0 = time.time()
et_final = ExtraTreesRegressor(
    n_estimators=500, max_features=0.5,
    min_samples_leaf=2, random_state=42, n_jobs=-1,
)
et_final.fit(X_tv, y_tv)
print(f"Training time     : {time.time() - t0:.1f}s")

y_pred_tv   = et_final.predict(X_tv)
y_pred_test = et_final.predict(X_test)

print()
print("=" * 70)
print("Final evaluation")
print("=" * 70)
evaluate(y_tv,   y_pred_tv,   'trainval')
evaluate(y_test, y_pred_test, 'TEST')

## 6. Feature Importance

In [ ]:
imp = pd.Series(et_final.feature_importances_, index=final_feature_names).sort_values(ascending=False)
top30 = imp.head(30)

fig, ax = plt.subplots(figsize=(10, 8))
colors = ['#1f77b4' if 'lv_' in n else
          '#2ca02c' if any(x in n for x in ['slope', 'accel', 'dil']) else
          '#ff7f0e' for n in top30.index]
ax.barh(top30.index[::-1], top30.values[::-1], color=colors[::-1])
ax.set_xlabel('Feature importance')
ax.set_title('Top 30 features by importance')

# Legend patches
from matplotlib.patches import Patch
legend_elements = [
    Patch(facecolor='#2ca02c', label='Slope features'),
    Patch(facecolor='#1f77b4', label='Latent vector features'),
    Patch(facecolor='#ff7f0e', label='Aggregate features'),
]
ax.legend(handles=legend_elements, fontsize=9, loc='lower right')

plt.tight_layout()
plt.show()

print(f"Cumulative importance — top 10: {top30.head(10).sum()*100:.1f}%  "
      f"top 30: {top30.sum()*100:.1f}%")

In [ ]:
# Importance breakdown by feature group
groups = {
    'Latent vectors': [n for n in final_feature_names if n.startswith('lv_')],
    'Slope features': [n for n in final_feature_names
                       if any(x in n for x in ['slope', 'accel', 'pred_log_ratio'])],
    'Aggregate features': [n for n in final_feature_names
                           if any(x in n for x in ['_agg_', 'dil_factor', 'is_box_iso'])],
    'Other': [],
}
covered = set()
for k, v in groups.items():
    covered |= set(v)
groups['Other'] = [n for n in final_feature_names if n not in covered]

print("Feature group importances:")
for group, names in groups.items():
    total = imp[names].sum() * 100
    print(f"  {group:22s}: {len(names):3d} features  {total:5.1f}%")

## 7. Error Analysis

In [ ]:
# Residual analysis on test set
residuals = y_pred_test - y_test
smape_per_row = 2 * np.abs(y_pred_test - y_test) / (np.abs(y_pred_test) + np.abs(y_test)) * 100

fig, axes = plt.subplots(2, 2, figsize=(13, 9))

# Actual vs predicted
mn, mx = min(y_test.min(), y_pred_test.min()), max(y_test.max(), y_pred_test.max())
axes[0, 0].scatter(y_test, y_pred_test, alpha=0.3, s=8, color='steelblue', linewidths=0)
axes[0, 0].plot([mn, mx], [mn, mx], 'r--', linewidth=1)
axes[0, 0].set_xlabel('Actual time (min)')
axes[0, 0].set_ylabel('Predicted time (min)')
axes[0, 0].set_title('Actual vs Predicted (test)')

# Residual distribution
axes[0, 1].hist(residuals, bins=50, color='seagreen', edgecolor='white', linewidth=0.3)
axes[0, 1].axvline(0, color='red', linestyle='--', linewidth=1)
axes[0, 1].set_xlabel('Residual (pred − actual, min)')
axes[0, 1].set_ylabel('Count')
axes[0, 1].set_title(f'Residual distribution  (mean={residuals.mean():.2f}, std={residuals.std():.2f})')

# SMAPE vs actual time
axes[1, 0].scatter(y_test, smape_per_row, alpha=0.3, s=8, color='darkorange', linewidths=0)
axes[1, 0].axhline(smape_per_row.mean(), color='red', linestyle='--', linewidth=1,
                   label=f'mean {smape_per_row.mean():.1f}%')
axes[1, 0].set_xlabel('Actual time (min)')
axes[1, 0].set_ylabel('Per-row SMAPE (%)')
axes[1, 0].set_title('Error vs actual time')
axes[1, 0].legend(fontsize=9)

# SMAPE CDF
sorted_smape = np.sort(smape_per_row)
cdf = np.arange(1, len(sorted_smape) + 1) / len(sorted_smape)
axes[1, 1].plot(sorted_smape, cdf, color='mediumpurple', linewidth=1.5)
axes[1, 1].axvline(smape_per_row.mean(), color='red', linestyle='--', linewidth=1,
                   label=f'mean {smape_per_row.mean():.1f}%')
for pct, val in [(50, np.percentile(smape_per_row, 50)),
                 (90, np.percentile(smape_per_row, 90))]:
    axes[1, 1].axvline(val, color='gray', linestyle=':', linewidth=1,
                       label=f'p{pct}={val:.1f}%')
axes[1, 1].set_xlabel('SMAPE (%)')
axes[1, 1].set_ylabel('CDF')
axes[1, 1].set_title('SMAPE CDF (test)')
axes[1, 1].legend(fontsize=9)

plt.tight_layout()
plt.show()

In [ ]:
# Worst predictions
test_rows = df_test[['Film', 'Developer', 'Dilution', 'ASA/ISO', '35mm']].copy()
test_rows['predicted'] = y_pred_test.round(2)
test_rows['smape']     = smape_per_row.round(2)
test_rows['residual']  = residuals.round(2)

print("Worst 15 predictions by SMAPE (test set):")
display(test_rows.sort_values('smape', ascending=False).head(15))

In [ ]:
# Error breakdown by stops bucket
stops_test = df_test['stops'].values
bins = [-np.inf, -1.5, -0.5, 0.5, 1.5, 2.5, np.inf]
labels = ['< -1.5', '-1.5–-0.5', '-0.5–0.5', '0.5–1.5', '1.5–2.5', '> 2.5']
bucket = pd.cut(stops_test, bins=bins, labels=labels)

bucket_df = pd.DataFrame({
    'bucket': bucket,
    'smape': smape_per_row,
    'mae': np.abs(residuals),
    'n': 1,
})

summary = bucket_df.groupby('bucket', observed=True).agg(
    n=('n', 'sum'),
    smape_mean=('smape', 'mean'),
    smape_median=('smape', 'median'),
    mae_mean=('mae', 'mean'),
).reset_index()

print("Error breakdown by push/pull bucket:")
display(summary.round(2))

In [ ]:
# Final performance summary
cv_smape_mean = cv_df['smape'].mean()
cv_smape_std  = cv_df['smape'].std()
test_smape    = smape(y_test, y_pred_test)
test_mae      = np.mean(np.abs(y_test - y_pred_test))

print("=" * 50)
print("Final Performance Summary")
print("=" * 50)
print(f"  CV SMAPE  : {cv_smape_mean:.2f}% ± {cv_smape_std:.2f}%")
print(f"  Test SMAPE: {test_smape:.2f}%")
print(f"  Test MAE  : {test_mae:.2f} min")
print(f"  Features  : {X_tv.shape[1]}")